In [1]:
import requests
import json
import secrets
 
client_info_fp = 'client_info.json'

In [2]:
with open(client_info_fp) as file:
    client_info = json.load(file)
CLIENT_ID = client_info['CLIENT_ID']
CLIENT_SECRET = client_info['CLIENT_SECRET']

In [3]:
#PKCE verifier code
def get_new_code_verifier() -> str:
   token = secrets.token_urlsafe(100)
   return token[:128] #128 characters


In [4]:
#user authorization link (generated code for cell below) 
def print_auth_link(code_verifier: str):
    url = f'https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id={CLIENT_ID}&code_challenge={code_verifier}'
    print(f"Authorize Here : {url}\n")

In [5]:
def request_user_token(auth_code : str, code_verifier: str) -> dict:
    url = 'https://myanimelist.net/v1/oauth2/token'
    data = {"client_id": CLIENT_ID,
            "client_secret": CLIENT_SECRET,
            "code": auth_code,
            "code_verifier": code_verifier,
            "grant_type": "authorization_code"
           }
    resp = requests.post(url, data = data)
    resp.raise_for_status() #if 400 reponse error may need to generate new code with above link
    token = resp.json()
    resp.close()
    print('Token generated')
    
    with open('token.json', 'w') as file:
            json.dump(token, file, indent=4)
            print('Token saved in "token.json"')
    return token

In [ ]:
def print_user_info(access_token: str):
    url = 'https://api.myanimelist.net/v2/users/@me'
    response = requests.get(url, headers = {
        'Authorization': f'Bearer {access_token}'
        })
    
    response.raise_for_status()
    user = response.json()
    response.close()

    print(f"\n>>> Greetings {user['name']}! <<<")



In [ ]:
code_verifier = get_new_code_verifier()
print_auth_link(code_verifier)

authorisation_code = input('Copy-paste the Authorisation Code: ').strip()
token = request_user_token(authorisation_code, code_verifier)

print_user_info(token['access_token'])
